# Kalman Filter

Kalman filters are linear models for state estimation of dynamic systems [1].  They have been the <i>de facto</i> standard in many robotics and tracking/prediction applications because they are well suited for systems with uncertainty about an observable dynamic process.  They use a "observe, predict, correct" paradigm to extract information from an otherwise noisy signal. In Pyro, we can build Kalman filters using the [`pyro.contrib.tracking` library](http://docs.pyro.ai/en/dev/contrib.tracking.html#module-pyro.contrib.tracking.extended_kalman_filter)

## Dynamic process

To start, consider this simple motion model:

$$ X_{k+1} = FX_k + \mathbf{W}_k $$
$$ \mathbf{Z}_k = HX_k + \mathbf{V}_k $$

where $k$ is the state, $X$ is the signal estimate, $Z_k$ is the observed value at timestep $k$, $\mathbf{W}_k$ and  $\mathbf{V}_k$ are independent noise processes (ie $\mathbb{E}[w_k v_j^T] = 0$ for all $j, k$) which we'll approximate as Gaussians. Note that the state transitions are linear.

## Kalman Update
At each time step, we perform a prediction for the mean and covariance:
$$ \hat{X}_k = F\hat{X}_{k-1}$$
$$\hat{P}_k = FP_{k-1}F^T + Q$$
and a correction for the measurement:
$$ K_k = \hat{P}_k H^T(H\hat{P}_k H^T + R)^{-1}$$
$$ X_k = \hat{X}_k + K_k(z_k - H\hat{X}_k)$$
$$ P_k = (I-K_k H)\hat{P}_k$$

where $X$ is the position estimate, $P$ is the covariance matrix, $K$ is the Kalman Gain, and $Q$ and $R$ are covariance matrices.

For an in-depth derivation, see \[1\]

## Nonlinear Estimation: Extended Kalman Filter

What if our system is non-linear, eg in GPS navigation?  Consider this non-linear system:

$$ X_{k+1} = \mathbf{f}(X_k) + \mathbf{W}_k $$
$$ \mathbf{Z}_k = \mathbf{h}(X_k) + \mathbf{V}_k $$

Notice that $\mathbf{f}$ and $\mathbf{h}$ are now (smooth) non-linear functions.

As an example, let's look at an object moving at near-constant velocity in 2-D in a discrete time space over 100 time steps.

In [1]:
import os
import math

import torch
import pyro
from pyro.distributions import MultivariateNormal
from pyro.contrib.autoguide import AutoDelta
from pyro.optim import Adam
from pyro.infer import SVI, TraceEnum_ELBO, config_enumerate
from pyro.contrib.tracking.extended_kalman_filter import EKFState
from pyro.contrib.tracking.distributions import EKFDistribution
from pyro.contrib.tracking.dynamic_models import NcvContinuous
from pyro.contrib.tracking.measurements import PositionMeasurement

smoke_test = ('CI' in os.environ)
pyro.enable_validation(True)

In [2]:
# Discrete time
dt = 1. / 100
num_frames = 10
dim = 4

# Continuous model
ncv = NcvContinuous(dimension=dim, sa2=2.0)

# Truth trajectory
xs_truth = torch.zeros(num_frames, dim)
# initial directory
theta0_truth = 0.0
# initial state
xs_truth[0, :] = torch.tensor([0.0, 0.0, math.cos(theta0_truth), math.sin(theta0_truth)])
for frame_num in range(1, num_frames):
    # sample independent process noise
    dx = pyro.sample('process_noise_{}'.format(frame_num), ncv.process_noise_dist(dt))
    xs_truth[frame_num, :] = ncv(xs_truth[frame_num-1, :], dt=dt) + dx

In [3]:
# Measurements
measurements = []
mean = torch.zeros(2)
# no correlations
cov = 1e-5 * torch.eye(2)
# sample independent measurement noise
dzs = pyro.sample('dzs', MultivariateNormal(mean, cov).expand((num_frames,)))
# compute measurement means
zs = xs_truth[:, :2] + dzs
# for i in range(len(zs)):
#     measurements.append(PositionMeasurement(zs[i], cov, frame_num=frame_num))
#     frame_num += 1
# measurements

In [4]:
dzs.shape

torch.Size([10, 2])

In [5]:
zs

tensor([[ 0.0027, -0.0032],
        [ 0.0105,  0.0047],
        [ 0.0160, -0.0019],
        [ 0.0332,  0.0011],
        [ 0.0372, -0.0017],
        [ 0.0442, -0.0018],
        [ 0.0619, -0.0034],
        [ 0.0624,  0.0001],
        [ 0.0782, -0.0002],
        [ 0.0916,  0.0004]], grad_fn=<ThAddBackward>)

In [6]:
xs_truth

tensor([[ 0.0000,  0.0000,  1.0000,  0.0000],
        [ 0.0100,  0.0000,  1.0040,  0.0096],
        [ 0.0200,  0.0001,  0.9893,  0.0104],
        [ 0.0298,  0.0003,  0.9775,  0.0229],
        [ 0.0396,  0.0005,  0.9694,  0.0104],
        [ 0.0493,  0.0006,  0.9522,  0.0210],
        [ 0.0587,  0.0009,  0.9487,  0.0189],
        [ 0.0683,  0.0011,  0.9770,  0.0115],
        [ 0.0781,  0.0011,  0.9832, -0.0085],
        [ 0.0880,  0.0010,  1.0000, -0.0240]], grad_fn=<CopySlices>)

In [7]:
@config_enumerate
def model(data):
    R = pyro.sample('R', dist.LogNormal(1., 1.)) * torch.eye(3)
    Q = pyro.sample('Q', dist.LogNormal(1., 1.)) * torch.eye(3)
    pyro.sample('track_{}'.format(i), EKFDistribution(xs_truth[0], R,
                                                      ncv, Q, time_steps=num_frames),
                obs=data)
    # observe the measurements
    
guide = AutoDelta(model)

The Extended Kalman Filter (EKF) attacks this problem by using a local linearization of the Kalman filter via a [Taylors Series expansion](https://en.wikipedia.org/wiki/Taylor_series).

$$ f(X_k, k) \approx f(x_k^R, k) + \mathbf{H}_k(X_k - x_k^R) + \cdots$$

where $\mathbf{H}_k$ is the Jacobian matrix at time $k$, $x_k^R$ is the previous optimal estimate, and we ignore the higher order terms.  At each time step, we compute a Jacobian conditioned the previous predictions (this computation is handled for us under the hood), and use the result to perform a prediction and update.

Omitting the derivations, the modification to the above predictions are now:
$$ \hat{X}_k \approx \mathbf{f}(X_{k-1}^R)$$
$$ \hat{P}_k = \mathbf{H}_\mathbf{f}(X_{k-1})P_{k-1}\mathbf{H}_\mathbf{f}^T(X_{k-1}) + Q$$
and the updates are now:
$$ X_k \approx \hat{X}_k + K_k\big(z_k - \mathbf{h}(\hat{X}_k)\big)$$
$$ K_k = \hat{P}_k \mathbf{H}_\mathbf{h}(\hat{X}_k) \Big(\mathbf{H}_\mathbf{h}(\hat{X}_k)\hat{P}_k \mathbf{H}_\mathbf{h}(\hat{X}_k) + R_k\Big)^{-1} $$
$$ P_k = \big(I - K_k \mathbf{H}_\mathbf{h}(\hat{X}_k)\big)\hat{P}_K$$

In Pyro, all we need to do is create an `EKFState` object and use its `predict` and `update` methods.

In [8]:
optim = pyro.optim.Adam({'lr': 0.2})
svi = SVI(model, guide, optim, loss=TraceEnum_ELBO(max_iarange_nesting=1))

pyro.set_rng_seed(0)
pyro.clear_param_store()

for i in range(100 if not smoke_test else 2):
    loss = svi.step(zs)
    print('loss: ', loss)

RuntimeError: size mismatch, m1: [2 x 4], m2: [3 x 3] at /Users/soumith/miniconda2/conda-bld/pytorch_1532623076075/work/aten/src/TH/generic/THTensorMath.cpp:2070

## References
[1] H. W. Sorenson, editor. Kalman filtering: theory and application. IEEE Press, 1985.

[2] J. K. Uhlmann. Algorithms for multiple target tracking. American Scientist, 80(2):128–141, 1992